<a href="https://colab.research.google.com/github/andresenrique08/ML_2020/blob/master/Proyecto_Malware/EntrenarModelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importar librerias

# Importing libraries
import numpy as np
import pydrive as drive
from google.colab import drive
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
pd.set_option('display.max_rows', 100)

# Modulos para contingencias
from scipy.stats import chi2
from scipy.stats import chi2_contingency

# Modulos de scikit-learn
from sklearn.preprocessing import MinMaxScaler
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ruta_train = '/content/drive/MyDrive/ProyectoFinalML/train_clear.csv'

train  = pd.read_csv(ruta_train, sep=",", low_memory=True) 
train = train.drop(['Unnamed: 0'], axis=1)
train.head()

,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IeVerIdentifier,Census_MDC2FormFactor,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSBranch,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_ActivationChannel,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsTouchEnabled,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0.556950,0.537115,0.596314,0.758125,0.0,0.126697,0.115254,0.602837,0.895716,0.000000,1.0,0.534936,0.844230,0.317757,1.00000,0.434082,0.026405,0.023622,0.444444,0.522448,5.844540e-08,1.000000,0.013081,0.002350,0.524783,0.105324,0.117259,0.104270,1.000000,1.000000e+00,0.569549,0.527209,0.007603,0.511569,0.512364,1.000000,0.657895,0.732919,0.393503,0.221906,0.574312,0.501200,0.0,0.0,0.0,0.642857,0.0
1,0.401938,0.496127,0.410464,0.758125,0.0,0.416290,0.400000,0.223404,0.895716,0.000000,1.0,0.534936,0.844230,0.317757,0.95943,0.434082,0.265285,0.023622,0.444444,0.536743,5.844540e-08,1.000000,0.004473,0.002350,0.502297,0.076389,0.111238,0.088994,0.916717,2.328306e-10,0.534535,0.527209,0.000046,0.511569,0.512364,0.895791,0.184211,0.186335,0.393503,0.221906,0.574312,0.802386,0.0,0.0,0.0,0.500000,0.0
2,0.556950,0.537115,0.533078,0.758125,0.0,0.384615,0.213559,0.170213,0.895716,0.969697,1.0,0.534936,0.746209,0.317757,1.00000,0.798828,0.919545,0.023622,0.444444,0.440027,1.402780e-08,0.922203,0.004976,0.002350,0.524783,0.120370,0.156319,0.125101,1.000000,1.000000e+00,0.569549,0.527209,0.007603,0.475018,0.475023,1.000000,0.157895,0.180124,1.000000,0.492814,0.128440,0.730592,0.0,0.0,0.0,0.142857,0.0
3,0.556950,0.537115,0.701773,0.758125,0.0,0.393665,0.393220,0.404255,0.895716,0.000000,1.0,0.534936,0.844230,0.317757,1.00000,0.234701,0.798528,0.023622,0.444444,0.507259,2.922331e-08,0.204658,0.009921,0.002350,0.525155,0.103009,0.111238,0.088994,1.000000,1.000000e+00,0.538361,0.527209,0.010505,0.511569,0.512364,1.000000,0.421053,0.391304,1.000000,0.492814,0.323853,0.277967,0.0,0.0,0.0,0.142857,1.0
4,0.556950,0.537115,0.527328,0.758125,0.0,0.076923,0.935593,0.262411,0.895716,0.969697,1.0,0.534936,0.746209,0.317757,0.95943,0.234701,0.960726,0.023622,0.444444,0.557963,5.844540e-08,1.000000,0.004451,0.003653,0.479965,0.076968,0.111238,0.088994,0.916717,0.000000e+00,0.488941,0.527209,0.008801,0.475018,0.475023,0.187473,0.184211,0.186335,1.000000,0.221906,0.323853,0.275109,0.0,0.0,0.0,0.000000,1.0


In [5]:
train.describe()

,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IeVerIdentifier,Census_MDC2FormFactor,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSBranch,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_ActivationChannel,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsTouchEnabled,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06,7.659039e+06
mean,5.016512e-01,5.016512e-01,5.016512e-01,6.771519e-01,5.716916e-02,4.851867e-01,5.718901e-01,4.319465e-01,7.795943e-01,6.191962e-01,8.601293e-01,5.016512e-01,7.818303e-01,2.868468e-01,9.534788e-01,3.581122e-01,6.900467e-01,2.348137e-02,3.920571e-01,5.272417e-01,4.290182e-07,9.280948e-01,1.678190e-02,3.615599e-03,5.016512e-01,9.209017e-02,1.261577e-01,1.038058e-01,9.076229e-01,2.531263e-01,5.016512e-01,5.016512e-01,4.574583e-02,5.016512e-01,5.016512e-01,5.818890e-01,3.563367e-01,3.674948e-01,5.648149e-01,2.914324e-01,3.687207e-01,4.579158e-01,5.072511e-01,1.309038e-01,2.930920e-01,4.908132e-01,5.016512e-01
std,6.609164e-02,5.759185e-02,7.809304e-02,2.013509e-01,8.850471e-02,2.854611e-01,3.024099e-01,2.461759e-01,1.817158e-01,4.658642e-01,1.469635e-01,4.874228e-02,5.649784e-02,7.853783e-02,6.935800e-02,2.112278e-01,2.072391e-01,1.577636e-02,1.433856e-01,1.873780e-01,5.870069e-04,2.054752e-01,1.419684e-02,3.019941e-03,2.849333e-02,3.373845e-02,2.946541e-02,2.463212e-02,1.224916e-01,4.348027e-01,4.270749e-02,3.100578e-02,1.366151e-01,2.319963e-02,2.296852e-02,3.779864e-01,2.676185e-01,2.786515e-01,4.163252e-01,1.640810e-01,2.028148e-01,2.940936e-01,4.999475e-01,3.372951e-01,4.551803e-01,3.252805e-01,4.999973e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.920674e-01,4.926996e-01,4.666667e-01,6.880914e-01,0.000000e+00,2.262443e-01,2.983051e-01,2.588652e-01,7.011462e-01,0.000000e+00,8.149000e-01,4.880950e-01,7.462088e-01,2.570093e-01,9.594302e-01,2.347005e-01,5.486473e-01,7.874016e-03,4.444444e-01,4.458343e-01,2.992462e-08,9.222034e-01,5.282185e-03,2.350451e-03,5.022970e-01,7.638889e-02,1.112377e-01,8.899433e-02,9.167168e-01,0.0

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop(['HasDetections'], axis=1), train['HasDetections'], test_size=0.8, train_size=0.2, random_state=42)
del train

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
score=clf.score(X_test, y_test)
print(score)

0.5527584070588482


In [7]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(objective='binary')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6333061976435689

In [12]:
!pip install catboost &> /dev/null
!pip install ipywidgets &> /dev/null
!jupyter nbextension enable --py widgetsnbextension &> /dev/null
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

categorical_features_indices = np.where(X_train.dtypes != np.float)[0]
model = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent',
    task_type = 'GPU'
)
model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    logging_level='Verbose',  # you can uncomment this for text output
    plot=True
)

model.score(X_test, y_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.038175
0:	learn: 0.6909069	test: 0.6909133	best: 0.6909133 (0)	total: 14.9ms	remaining: 14.9s
1:	learn: 0.6888334	test: 0.6888370	best: 0.6888370 (1)	total: 28.8ms	remaining: 14.4s
2:	learn: 0.6868971	test: 0.6869067	best: 0.6869067 (2)	total: 42.5ms	remaining: 14.1s
3:	learn: 0.6851207	test: 0.6851311	best: 0.6851311 (3)	total: 61.2ms	remaining: 15.2s
4:	learn: 0.6834428	test: 0.6834602	best: 0.6834602 (4)	total: 79.4ms	remaining: 15.8s
5:	learn: 0.6819038	test: 0.6819260	best: 0.6819260 (5)	total: 93ms	remaining: 15.4s
6:	learn: 0.6804520	test: 0.6804733	best: 0.6804733 (6)	total: 108ms	remaining: 15.3s
7:	learn: 0.6790700	test: 0.6790864	best: 0.6790864 (7)	total: 121ms	remaining: 15s
8:	learn: 0.6777854	test: 0.6777961	best: 0.6777961 (8)	total: 134ms	remaining: 14.8s
9:	learn: 0.6765878	test: 0.6766010	best: 0.6766010 (9)	total: 148ms	remaining: 14.6s
10:	learn: 0.6754569	test: 0.6754660	best: 0.6754660 (10)	total: 161ms	remaining: 14.5s
11:	learn: 0.6744192

0.6352534064321377